In [1]:
import os.path
import numpy as np

# Load in data, this assumed you have a folder in env named data
data_dir = "data"
X_test = np.load(os.path.join(data_dir, "X_test.npy"))
y_test = np.load(os.path.join(data_dir, "y_test.npy"))
person_test = np.load(os.path.join(data_dir, "person_test.npy")).squeeze(axis=1)
X_train_valid = np.load(os.path.join(data_dir, "X_train_valid.npy"))
y_train_valid = np.load(os.path.join(data_dir, "y_train_valid.npy"))
person_train_valid = np.load(os.path.join(data_dir, "person_train_valid.npy")).squeeze(axis=1)

# Predefine some useful variables and fix data a bit
n_class = len(set(y_train_valid))
n_trials = 5
min_y = min(y_train_valid)
y_train_valid = y_train_valid - min_y
y_test = y_test - min_y

In [2]:
# Validate data loaded in correctly and print shapes
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'. format (X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115,)
Person test shape: (443,)


In [3]:
import random
import torch

# Define random seed so that we can reproduce results
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# This is for macbook M1, if you have intel I think you use cuda not mps
# Research what works for your device and change the torch.device
device = torch.device("mps")
device

device(type='mps')

In [4]:
# Import the models and functions
from models import *
from trainer import *

In [5]:
# Set up data sets
s1_indices_train = [i for i, s in enumerate(person_train_valid) if s == 1]
s1_indices_test = [i for i, s in enumerate(person_test) if s == 1]
X_train_valid_s1 = X_train_valid[s1_indices_train]
y_train_valid_s1 = y_train_valid[s1_indices_train]
X_test_s1 = X_test[s1_indices_test]
y_test_s1 = y_test[s1_indices_test]

# Make dataloader test set for the single subject
# Convert data to tensors
X_tensor = torch.FloatTensor(X_test_s1)
y_tensor = torch.LongTensor(y_test_s1)

# Combine X and y into a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# Prepare dataloaders
test_dataloader = DataLoader(dataset, batch_size=256, shuffle=False)

# Train for the single subject data
Currently using low epochs just for testing purposes

## CNN

In [ ]:
valid_accs = []
test_accs = []

for i in range(n_trials):
    cnn = CNN(input_size=X_train_valid_s1.shape[1:], N=n_class).to(device)
    valid_acc = fit(cnn, X_train_valid_s1, y_train_valid_s1, device, epochs=10)
    test_acc, _ = evaluate(cnn, test_dataloader, device)
    valid_accs.append(valid_acc)
    test_accs.append(test_acc)

round(np.mean(valid_accs), 5), round(np.mean(test_accs), 5)

## RNN (LSTM)

In [ ]:
valid_accs = []
test_accs = []

for i in range(n_trials):
    rnn = LSTM().to(device)
    valid_acc = fit(rnn, X_train_valid_s1, y_train_valid_s1, device, epochs=10)
    test_acc, _ = evaluate(rnn, test_dataloader, device)
    valid_accs.append(valid_acc)
    test_accs.append(test_acc)

round(np.mean(valid_accs), 5), round(np.mean(test_accs), 5)

## Another RNN (GRU)

GRU takes super long to train, not entirely sure why

I even cut out a fc layer to speed it up, and increase lr

In [ ]:
valid_accs = []
test_accs = []

for i in range(n_trials):
    gru = GRU().to(device)
    valid_acc = fit(gru, X_train_valid_s1, y_train_valid_s1, device, epochs=10)
    test_acc, _ = evaluate(gru, test_dataloader, device)
    valid_accs.append(valid_acc)
    test_accs.append(test_acc)

round(np.mean(valid_accs), 5), round(np.mean(test_accs), 5)

# CNN + RNN (LSTM)

In [6]:
valid_accs = []
test_accs = []

for i in range(n_trials):
    crnn = CNN_RNN(output_size=n_class,).to(device)
    valid_acc = fit(crnn, X_train_valid_s1, y_train_valid_s1, device, epochs=10)
    test_acc, _ = evaluate(crnn, test_dataloader, device)
    valid_accs.append(valid_acc)
    test_accs.append(test_acc)

round(np.mean(valid_accs), 5), round(np.mean(test_accs), 5)

  0%|          | 0/10 [00:00<?, ?it/s]

Best valid accuracy: 0.27083


  0%|          | 0/10 [00:00<?, ?it/s]